In [ ]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd

def get_db_connection():
    server = SSHTunnelForwarder(
        ('letsrise.myonline.works', 22),  # SSH server and port
        ssh_username='ubuntu',  # SSH username
        ssh_pkey='~/.ssh/id_letsrise',  # Path to your private key
        remote_bind_address=('localhost', 5432)  # Database server and port
    )

    server.start()

    conn = psycopg2.connect(
        dbname='letsrise_v1',
        user='letsrise_intern',
        password='letsrise',
        host='localhost',
        port=server.local_bind_port  # Use the dynamically assigned local port
    )
    return conn, server

In [ ]:
conn, server = get_db_connection()
query1 = """
SELECT * FROM public.user_info ui
INNER JOIN public.assessment_entries ae ON ui.user_id = ae.user_id
INNER JOIN public.consequence_results cr ON ui.user_id = cr.user_id;
"""
query2 = """
SELECT crd.*, ui.name, b.benchmark_name
FROM public.comparison_result_data crd
INNER JOIN public.user_info ui ON crd.user_id = ui.user_id
INNER JOIN public.benchmark b ON crd.benchmark_id = b.benchmark_id
ORDER BY crd.comparison_id ASC;
"""

consequence_df = pd.read_sql(query1, conn)
comparison_df = pd.read_sql(query2, conn)
conn.close()
server.stop()



In [ ]:
comparison_df.head(5)

In [ ]:
consequence_df = consequence_df.loc[:, ~consequence_df.columns.duplicated()]
required_columns = ['user_id', 'name', 'email', 'age', 'linkedin_url', 'education_level', 'employment_status', 'entrepreneurial_experience', 'current_startup_stage', 'number_of_startups', 'role_in_entrepreneurship', 'industry_experience', 'number_of_previous_startups', 'location', 'gender', 'startup_name', 'assessment_id', 
                    'customer_centric', 'collaborative', 'agile', 'innovative', 'risk_taking', 'visionary', 'hustler', 'passionate', 'resilient', 'educational', 'analytical', 'frugal', 'legacy', 'digital', 'problem_solver', 
                    'delayed_product_market_fit', 'lack_of_product_market_fit', 'unable_to_complete_fundraise', 'lack_of_growth', 'lack_of_revenue', 'high_turnover_of_talent', 'inefficient_processes', 'time_consuming_client_acquisition', 'low_customer_conversion', 'low_customer_satisfaction', 'low_customer_retention', 'high_cash_burnrate', 'high_team_conflict', 'high_key_man_risk', 'lack_of_partnerships_collaborations', 'lack_of_scalability', 'lack_of_data_integrity', 'lack_of_data_security', 'lack_of_marketing', 'lack_of_motivation', 'lack_of_leadership', 'lack_of_innovation', 'lack_of_clarity', 'too_much_dependency_on_external_factors', 'lack_of_technological_advancements', 'lack_of_unique_value_proposition', 'lack_of_customer_variety', 'lack_of_intellectual_property', 'lacking_solution_quality', 'lack_of_supporters', 'missed_opportunities', 'delayed_revenue']
filtered_df = consequence_df[required_columns]
filtered_df.head(5)

In [ ]:
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Define the traits to be plotted
traits = ['customer_centric', 'collaborative', 'agile', 'innovative', 'risk_taking', 'visionary', 'hustler', 'passionate', 'resilient', 'educational', 'analytical', 'frugal', 'legacy', 'digital', 'problem_solver']

# Initialize the Dash app
app = Dash(__name__)

# Get the list of benchmarks and users for the dropdowns
benchmarks = comparison_df['benchmark_name'].unique()
users = comparison_df[['user_id', 'name']].drop_duplicates().to_dict('records')

# Define the layout of the app
app.layout = html.Div([
    html.H1("Benchmark Comparison", style={'text-align': 'center', 'color': 'var(--primary-color)', 'font-family': 'var(--font-family)'}),
    html.Div([
        dcc.Dropdown(
            id='benchmark-dropdown',
            options=[{'label': benchmark, 'value': benchmark} for benchmark in benchmarks],
            placeholder="Select a benchmark",
            style={'width': '40%', 'display': 'inline-block', 'margin-right': '20px', 'font-family': 'var(--font-family)', 'color': 'var(--text-color)'}
        ),
        dcc.Dropdown(
            id='user-dropdown',
            options=[{'label': user['name'], 'value': user['user_id']} for user in users],
            placeholder="Select a user",
            style={'width': '40%', 'display': 'inline-block', 'font-family': 'var(--font-family)', 'color': 'var(--text-color)'}
        )
    ], style={'text-align': 'center', 'margin-bottom': '20px'}),
    dcc.Graph(id='benchmark-graph')
], style={'background-color': 'var(--background-color)', 'font-family': 'var(--font-family)', 'padding': '20px'})

# Define the callback to update the benchmark graph based on selected benchmark and user
@app.callback(
    Output('benchmark-graph', 'figure'),
    [Input('benchmark-dropdown', 'value'), Input('user-dropdown', 'value')]
)
def update_benchmark_graph(selected_benchmark, selected_user):
    if not selected_benchmark or not selected_user:
        return px.bar(title="Please select both a benchmark and a user to see the comparison.")
    
    # Filter the data for the selected benchmark and user
    filtered_data = comparison_df[(comparison_df['benchmark_name'] == selected_benchmark) & (comparison_df['user_id'] == selected_user)]
    
    if filtered_data.empty:
        return px.bar(title="No data available for the selected benchmark and user.")
    
    # Create a vertical bar chart for the traits
    trait_data = filtered_data[traits].melt()
    fig = px.bar(trait_data, x='variable', y='value', title=f'Benchmark Comparison for {selected_benchmark}', 
                 labels={'variable': 'Trait', 'value': 'Score'}, color='variable', 
                 color_discrete_sequence=px.colors.qualitative.Set3)
    fig.update_layout(
        xaxis={'categoryorder':'total descending'},
        plot_bgcolor='var(--white-color)',
        paper_bgcolor='var(--background-color)',
        font=dict(family='var(--font-family)', color='var(--text-color)'),
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_title=None,
        yaxis_title="Score"
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


traits = ['customer_centric', 'collaborative', 'agile', 'innovative', 'risk_taking', 'visionary', 'hustler', 'passionate', 'resilient', 'educational', 'analytical', 'frugal', 'legacy', 'digital', 'problem_solver']
consequences = ['delayed_product_market_fit', 'lack_of_product_market_fit', 'unable_to_complete_fundraise', 'lack_of_growth', 'lack_of_revenue', 'high_turnover_of_talent', 'inefficient_processes', 'time_consuming_client_acquisition', 'low_customer_conversion', 'low_customer_satisfaction', 'low_customer_retention', 'high_cash_burnrate', 'high_team_conflict', 'high_key_man_risk', 'lack_of_partnerships_collaborations', 'lack_of_scalability', 'lack_of_data_integrity', 'lack_of_data_security', 'lack_of_marketing', 'lack_of_motivation', 'lack_of_leadership', 'lack_of_innovation', 'lack_of_clarity', 'too_much_dependency_on_external_factors', 'lack_of_technological_advancements', 'lack_of_unique_value_proposition', 'lack_of_customer_variety', 'lack_of_intellectual_property', 'lacking_solution_quality', 'lack_of_supporters', 'missed_opportunities', 'delayed_revenue']

# Initialize the Dash app
app = Dash(__name__)

# Get the list of users for the dropdown
users = filtered_df[['user_id', 'name']].drop_duplicates().to_dict('records')

# Function to format consequence names
def format_consequence_name(name):
    return name.replace('_', ' ').title()

formatted_consequences = [format_consequence_name(c) for c in consequences]

# Define the layout of the app
app.layout = html.Div([
    html.H1("User Trait Scores and Consequences Viewer", style={'text-align': 'center', 'color': 'var(--primary-color)', 'font-family': 'var(--font-family)'}),
    dcc.Dropdown(
        id='user-dropdown',
        options=[{'label': user['name'], 'value': user['user_id']} for user in users],
        placeholder="Select a user",
        style={'width': '50%', 'margin': 'auto', 'font-family': 'var(--font-family)', 'color': 'var(--text-color)'}
    ),
    html.Div(id='user-info', style={'text-align': 'center', 'margin': '20px', 'font-family': 'var(--font-family)', 'color': 'var(--text-color)'}),
    html.Div([
        dcc.Graph(id='trait-scores-graph', style={'margin-bottom': '40px'}),
        dcc.Graph(id='consequence-graph')
    ], style={'display': 'flex', 'flex-direction': 'column', 'align-items': 'center'})
], style={'background-color': 'var(--background-color)', 'font-family': 'var(--font-family)', 'padding': '20px'})

# Define the callback to update the user info and trait scores graph based on selected user
@app.callback(
    [Output('user-info', 'children'),
     Output('trait-scores-graph', 'figure')],
    [Input('user-dropdown', 'value')]
)
def update_trait_scores(selected_user):
    if not selected_user:
        return "Please select a user to see their information.", px.bar(title="Please select a user to see their trait scores.")
    
    # Filter the data for the selected user
    user_data = filtered_df[filtered_df['user_id'] == selected_user]
    trait_data = user_data[traits].melt()
    
    # Create a horizontal bar chart for the user's trait scores
    fig = px.bar(trait_data, y='variable', x='value', orientation='h', title=f'Trait Scores', 
                 labels={'variable': 'Trait', 'value': 'Score'}, color='variable', 
                 color_discrete_sequence=px.colors.qualitative.Set3)
    fig.update_layout(
        yaxis={'categoryorder':'total ascending'},
        plot_bgcolor='var(--white-color)',
        paper_bgcolor='var(--background-color)',
        font=dict(family='var(--font-family)', color='var(--text-color)'),
        margin=dict(l=150, r=50, t=50, b=50),
        yaxis_title=None,
        xaxis_title="Score"
    )

    # User info div
    user_info = html.Div([
        html.H2(user_data['name'].values[0], style={'color': 'var(--primary-color)'}),
        html.P(f"Email: {user_data['email'].values[0]}", style={'color': 'var(--text-color)'}),
        html.P(f"Age: {user_data['age'].values[0]}", style={'color': 'var(--text-color)'}),
        html.P(f"LinkedIn: {user_data['linkedin_url'].values[0]}", style={'color': 'var(--text-color)'})
    ])
    
    return user_info, fig

# Define the callback to update the consequence graph based on selected user
@app.callback(
    Output('consequence-graph', 'figure'),
    [Input('user-dropdown', 'value')]
)
def update_consequence(selected_user):
    if not selected_user:
        return px.bar(title="Please select a user to see their consequences.")
    
    # Filter the data for the selected user
    user_data = filtered_df[filtered_df['user_id'] == selected_user]
    consequence_data = user_data[consequences].melt()
    consequence_data = consequence_data[consequence_data['value'] > 0]
    
    if consequence_data.empty:
        return px.bar(title="No consequences with non-zero values for the selected user.")
    
    # Update the consequence_data variable names
    consequence_data['variable'] = consequence_data['variable'].apply(format_consequence_name)
    
    # Create a horizontal bar chart for the user's consequences
    fig = px.bar(consequence_data, y='variable', x='value', orientation='h', title=f'Consequences', 
                 labels={'variable': 'Consequence', 'value': 'Value'}, color='variable', 
                 color_discrete_sequence=px.colors.qualitative.Set3)
    fig.update_layout(
        yaxis={'categoryorder':'total ascending'},
        plot_bgcolor='var(--white-color)',
        paper_bgcolor='var(--background-color)',
        font=dict(family='var(--font-family)', color='var(--text-color)'),
        margin=dict(l=150, r=50, t=50, b=50),
        yaxis_title=None,
        xaxis_title="Value"
    )
    
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Define the traits, consequences, and initial user
traits = ['customer_centric', 'collaborative', 'agile', 'innovative', 'risk_taking', 'visionary', 'hustler', 'passionate', 'resilient', 'educational', 'analytical', 'frugal', 'legacy', 'digital', 'problem_solver']
consequences = ['delayed_product_market_fit', 'lack_of_product_market_fit', 'unable_to_complete_fundraise', 'lack_of_growth', 'lack_of_revenue', 'high_turnover_of_talent', 'inefficient_processes', 'time_consuming_client_acquisition', 'low_customer_conversion', 'low_customer_satisfaction', 'low_customer_retention', 'high_cash_burnrate', 'high_team_conflict', 'high_key_man_risk', 'lack_of_partnerships_collaborations', 'lack_of_scalability', 'lack_of_data_integrity', 'lack_of_data_security', 'lack_of_marketing', 'lack_of_motivation', 'lack_of_leadership', 'lack_of_innovation', 'lack_of_clarity', 'too_much_dependency_on_external_factors', 'lack_of_technological_advancements', 'lack_of_unique_value_proposition', 'lack_of_customer_variety', 'lack_of_intellectual_property', 'lacking_solution_quality', 'lack_of_supporters', 'missed_opportunities', 'delayed_revenue']

# Initialize the Dash app
app = Dash(__name__)

# Get the list of benchmarks and users for the dropdowns
benchmarks = comparison_df['benchmark_name'].unique()
users = comparison_df[['user_id', 'name']].drop_duplicates().to_dict('records')

# Function to format consequence names
def format_consequence_name(name):
    return name.replace('_', ' ').title()

formatted_consequences = [format_consequence_name(c) for c in consequences]

# Define the layout of the app
app.layout = html.Div([
    html.H1("Benchmark and Trait Scores Comparison", style={'text-align': 'center', 'color': 'var(--primary-color)', 'font-family': 'var(--font-family)', 'margin-bottom': '20px'}),
    html.Div(id='user-info', style={'text-align': 'center', 'margin': '20px', 'font-family': 'var(--font-family)', 'color': 'var(--text-color)'}),
    html.Div([
        dcc.Dropdown(
            id='benchmark-dropdown',
            options=[{'label': benchmark, 'value': benchmark} for benchmark in benchmarks],
            placeholder="Select a benchmark",
            value='Global',
            style={'width': '40%', 'display': 'inline-block', 'margin-right': '20px', 'font-family': 'var(--font-family)', 'color': 'var(--text-color)', 'margin-bottom': '20px'}
        ),
        dcc.Dropdown(
            id='user-dropdown',
            options=[{'label': user['name'], 'value': user['user_id']} for user in users],
            placeholder="Select a user",
            value='02e6fb77-58f9-4930-9ee3-47740bfe618c',  # Default value for "Shamimuzzaman Chowdhury"
            style={'width': '40%', 'display': 'inline-block', 'font-family': 'var(--font-family)', 'color': 'var(--text-color)', 'margin-bottom': '20px'}
        ),
    ], style={'text-align': 'center', 'margin-bottom': '20px'}),
    dcc.Graph(id='benchmark-graph', style={'margin-bottom': '40px', 'height': '300px'}),
    dcc.Graph(id='trait-scores-graph', style={'margin-bottom': '40px', 'height': '300px'}),
    dcc.Graph(id='consequence-graph', style={'margin-bottom': '40px', 'height': '300px'}),
], style={'background-color': 'var(--background-color)', 'font-family': 'var(--font-family)', 'padding': '20px', 'border': '1px solid var(--border-color)', 'box-shadow': '0 4px 8px var(--shadow-color)'})

@app.callback(
    Output('user-info', 'children'),
    [Input('user-dropdown', 'value')]
)
def update_user_info(selected_user):
    if not selected_user:
        return "Please select a user to see their information."
    
    # Filter the data for the selected user
    user_data = filtered_df[filtered_df['user_id'] == selected_user]
    
    if user_data.empty:
        return "No user data available."
    
    # User info div
    user_info = html.Div([
        html.H2(user_data['name'].values[0], style={'color': 'var(--primary-color)'}),
        html.P(f"Email: {user_data['email'].values[0]}", style={'color': 'var(--text-color)'}),
        html.P(f"Age: {user_data['age'].values[0]}", style={'color': 'var(--text-color)'}),
        html.P(f"LinkedIn: {user_data['linkedin_url'].values[0]}", style={'color': 'var(--text-color)'})
    ])
    
    return user_info

@app.callback(
    Output('benchmark-graph', 'figure'),
    [Input('benchmark-dropdown', 'value'), Input('user-dropdown', 'value')]
)
def update_benchmark_graph(selected_benchmark, selected_user):
    if not selected_benchmark or not selected_user:
        return px.bar(title="Please select both a benchmark and a user to see the comparison.")
    
    # Filter the data for the selected benchmark and user
    filtered_data = comparison_df[(comparison_df['benchmark_name'] == selected_benchmark) & (comparison_df['user_id'] == selected_user)]
    
    if filtered_data.empty:
        return px.bar(title="No data available for the selected benchmark and user.")
    
    # Create a vertical bar chart for the traits
    trait_data = filtered_data[traits].melt()
    fig = px.bar(trait_data, x='variable', y='value', title=f'Benchmark Comparison for {selected_benchmark}', 
                 labels={'variable': 'Trait', 'value': 'Score'}, color='variable', 
                 color_discrete_sequence=px.colors.qualitative.Set3)
    fig.update_layout(
        xaxis={'categoryorder':'total descending'},
        plot_bgcolor='var(--white-color)',
        paper_bgcolor='var(--background-color)',
        font=dict(family='var(--font-family)', color='var(--text-color)'),
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_title=None,
        yaxis_title="Score"
    )
    
    return fig

@app.callback(
    Output('trait-scores-graph', 'figure'),
    [Input('user-dropdown', 'value')]
)
def update_trait_scores(selected_user):
    if not selected_user:
        return px.bar(title="Please select a user to see their trait scores.")
    
    # Filter the data for the selected user
    user_data = filtered_df[filtered_df['user_id'] == selected_user]
    
    if user_data.empty:
        return px.bar(title="No trait scores available for the selected user.")
    
    trait_data = user_data[traits].melt()
    
    # Create a horizontal bar chart for the user's trait scores
    fig = px.bar(trait_data, y='variable', x='value', orientation='h', title=f'Trait Scores', 
                 labels={'variable': 'Trait', 'value': 'Score'}, color='variable', 
                 color_discrete_sequence=px.colors.qualitative.Set3)
    fig.update_layout(
        yaxis={'categoryorder':'total ascending'},
        plot_bgcolor='var(--white-color)',
        paper_bgcolor='var(--background-color)',
        font=dict(family='var(--font-family)', color='var(--text-color)'),
        margin=dict(l=150, r=50, t=50, b=50),
        yaxis_title=None,
        xaxis_title="Score"
    )
    return fig

@app.callback(
    Output('consequence-graph', 'figure'),
    [Input('user-dropdown', 'value')]
)
def update_consequence(selected_user):
    if not selected_user:
        return px.bar(title="Please select a user to see their consequences.")
    
    # Filter the data for the selected user
    user_data = filtered_df[filtered_df['user_id'] == selected_user]
    consequence_data = user_data[consequences].melt()
    consequence_data = consequence_data[consequence_data['value'] > 0]
    
    if consequence_data.empty:
        return px.bar(title="No consequences with non-zero values for the selected user.")
    
    # Update the consequence_data variable names
    consequence_data['variable'] = consequence_data['variable'].apply(format_consequence_name)
    
    # Create a horizontal bar chart for the user's consequences
    fig = px.bar(consequence_data, y='variable', x='value', orientation='h', title=f'Consequences', 
                 labels={'variable': 'Consequence', 'value': 'Value'}, color='variable', 
                 color_discrete_sequence=px.colors.qualitative.Set3)
    fig.update_layout(
        yaxis={'categoryorder':'total ascending'},
        plot_bgcolor='var(--white-color)',
        paper_bgcolor='var(--background-color)',
        font=dict(family='var(--font-family)', color='var(--text-color)'),
        margin=dict(l=100, r=50, t=50, b=50),
        yaxis_title=None,
        xaxis_title="Value"
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
